In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Aud_Jpy'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_2022-2023.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join='inner').max(axis=1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha=1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha=1/lookback).mean()

    return adx_smooth


df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,ema50,ema25,adx
0,2022-06-17 13:00:00,93.914,93.930,93.331,93.581,93.933,93.950,93.348,93.599,93.924,93.940,93.340,93.590,54800,93.534934,93.538888,93.547246,93.565595,20.189701
1,2022-06-17 17:00:00,93.583,93.690,93.476,93.566,93.599,93.717,93.494,93.709,93.591,93.698,93.486,93.638,28610,93.541870,93.546055,93.554774,93.573307,12.364988
2,2022-06-19 21:00:00,93.363,94.165,93.363,93.904,93.513,94.186,93.513,93.927,93.438,94.175,93.438,93.916,22475,93.565682,93.571470,93.583483,93.608761,10.031819
3,2022-06-20 01:00:00,93.908,93.996,93.371,93.905,93.932,94.018,93.395,93.924,93.920,94.007,93.383,93.914,34945,93.586721,93.593905,93.608734,93.639524,11.563742
4,2022-06-20 05:00:00,93.909,94.173,93.767,94.098,93.927,94.190,93.788,94.113,93.918,94.182,93.778,94.106,37100,93.616581,93.625980,93.645363,93.685441,12.170888


In [7]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,ema50,ema25,adx
1543,2023-06-14 13:00:00,95.057,95.276,95.017,95.134,95.074,95.293,95.034,95.150,95.066,95.284,95.026,95.142,24121,91.919335,92.813659,93.634037,94.298374,46.941941
1544,2023-06-14 17:00:00,95.128,95.241,94.587,95.193,95.144,95.260,94.608,95.213,95.136,95.250,94.598,95.203,45395,91.952009,92.860972,93.695565,94.367960,48.202791
1545,2023-06-14 21:00:00,95.229,95.313,95.059,95.306,95.379,95.425,95.128,95.324,95.304,95.350,95.120,95.315,13005,91.985471,92.909567,93.759072,94.440810,46.836873
1546,2023-06-15 01:00:00,95.306,96.169,95.154,96.161,95.325,96.183,95.192,96.174,95.316,96.176,95.173,96.168,28664,92.027089,92.974091,93.853540,94.573670,45.939715
1547,2023-06-15 05:00:00,96.161,96.397,96.060,96.190,96.176,96.413,96.076,96.204,96.168,96.405,96.069,96.197,31686,92.068580,93.037911,93.945440,94.698542,46.589689


In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [9]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [10]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(pips_to_risk, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    win_amounts, loss_amounts, pips_risked = [], [], []
    day_fees = 0
    trades = []
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk

    if n_bars == 'any' or not wait_for_close:
        use_pullback = False

    if n_bars == 'any':
       lookback, lookforward = 12, 0

    else:
        lookback = n_bars + 1 if use_pullback else n_bars
        lookforward = -1 if use_pullback else 0

        if not wait_for_close:
            lookback, lookforward = lookback - 1, 0

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_ask_close, curr_bid_close, curr_mid_high, curr_mid_low, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Ask_Close', 'Bid_Close', 'Mid_High', 'Mid_Low', 'Date']]
        within_time_slot = start_hour <= curr_date.hour <= end_hour

        if len(trades) == 0 and within_time_slot:
            spread = abs(curr_ask_open - curr_bid_open)
            buy_pips_offset, sell_pips_offset = 0, 0

            if n_bars == 'any':
                mid_opens = list(df.loc[df.index[i - lookback:i], 'Mid_Open'])
                mid_highs = list(df.loc[df.index[i - lookback:i], 'Mid_High'])
                mid_lows = list(df.loc[df.index[i - lookback:i], 'Mid_Low'])
                mid_closes = list(df.loc[df.index[i - lookback:i], 'Mid_Close'])

                def _check_bars(pips_moved, buy):
                    for j in range(len(mid_opens) - 1, -1, -1):
                        if (buy and mid_opens[j] < mid_closes[j]) or (not buy and mid_opens[j] > mid_closes[j]):
                            pips_moved += abs(mid_opens[j] - mid_closes[j])

                            if pips_moved >= pip_movement:
                                return True
                            
                        else:
                            return False
                            
                    return False
                
                def _get_pips_moved(buy):
                    moved = 0

                    for j in range(len(mid_opens) - 1, -1, -1):
                        if (buy and mid_opens[j] < mid_closes[j]) or (not buy and mid_opens[j] > mid_closes[j]):
                            moved += abs(mid_opens[j] - mid_closes[j])
                            
                        else:
                            return moved
                            
                    return moved

                if not wait_for_close:
                    buy_signal = _check_bars(abs(curr_mid_open - curr_mid_high), buy=True)
                    sell_signal = _check_bars(abs(curr_mid_open - curr_mid_low), buy=False)
                    buy_pips_offset = pip_movement - _get_pips_moved(buy=True)
                    sell_pips_offset = pip_movement - _get_pips_moved(buy=False)

                else:
                    buy_signal = _check_bars(0, buy=True)
                    sell_signal = _check_bars(0, buy=False)

            else:
                mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
                mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
                mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
                mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

                if not wait_for_close:
                    buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) + abs(curr_mid_open - curr_mid_high) >= pip_movement
                    sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) + abs(curr_mid_open - curr_mid_low) >= pip_movement
                    buy_pips_offset, sell_pips_offset = pip_movement - abs(mid_opens[0] - mid_closes[-1]), pip_movement - abs(mid_opens[0] - mid_closes[-1])

                else:
                    buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
                    sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

            if use_pullback:
                mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
                buy_signal = buy_signal and mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)
                sell_signal = sell_signal and mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

            if ema is not None:
                curr_ema, curr_mid_close = df.loc[df.index[i - 1], [ema, 'Mid_Close']]
                buy_signal = buy_signal and curr_mid_close > curr_ema
                sell_signal = sell_signal and curr_mid_close < curr_ema

            curr_adx = df.loc[df.index[i - 1], 'adx']
            moving_enough = curr_adx > adx_cutoff

            buy_signal = buy_signal and moving_enough
            sell_signal = sell_signal and moving_enough

            highest_high, lowest_low = max(mid_highs), min(mid_lows)
            buy_pips_offset, sell_pips_offset = max([buy_pips_offset, 0]), max([sell_pips_offset, 0])

            if buy_signal or sell_signal:
                open_price = float(curr_ask_open + buy_pips_offset)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units(curr_pips_to_risk, open_price - (spread / 2), currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                 'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                 'end_date': None, 'prev_profit_ratio': None}

                        trades.append(trade)

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

                open_price = float(curr_bid_open - sell_pips_offset)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units(curr_pips_to_risk, open_price + (spread / 2), currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                'end_date': None, 'prev_profit_ratio': None}

                        trades.append(trade)

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        trades_to_remove = []

        for trade in trades:
            if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trades_to_remove.append(trade)

                continue

            if trade['trade_type'] == 'buy' and use_trailing_sl and curr_bid_close > trade['open_price']:
                curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

                # Initial move
                if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                    trade['stop_loss'] = trade['open_price']
                    trade['prev_profit_ratio'] = 0.0

                # Subsequent moves
                if curr_profit_ratio >= 2.0:
                    while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                        trade['prev_profit_ratio'] += 1.0
                        trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

            if trade is not None and trade['trade_type'] == 'buy' and not use_trailing_sl and curr_bid_high >= trade['stop_gain']:
                trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trades_to_remove.append(trade)

                continue

            if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trades_to_remove.append(trade)

                continue

            if trade is not None and trade['trade_type'] == 'sell' and use_trailing_sl and curr_ask_close < trade['open_price']:
                curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

                # Initial move
                if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                    trade['stop_loss'] = trade['open_price']
                    trade['prev_profit_ratio'] = 0.0

                # Subsequent moves
                if curr_profit_ratio >= 2.0:
                    while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                        trade['prev_profit_ratio'] += 1.0
                        trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

            if trade is not None and trade['trade_type'] == 'sell' and not use_trailing_sl and curr_ask_low <= trade['stop_gain']:
                trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trades_to_remove.append(trade)

                continue

        for trade in trades_to_remove:
            trades.remove(trade)

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts,

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.1]
n_bars_vals = ['any']  # LEAVE THE 'any' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pip_movement_vals = [20, 30]
use_pullback_vals = [False]
pips_to_risk_vals = ['bars', 10, 20, 30]  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.0]
ema_vals = [None, 'ema200', 'ema100', 'ema50']  # 'None' means it won't use the EMA
adx_cutoffs = [0, 25, 30]  # ADX is between 0 and 100; usually ADX > 25 means the market is moving fairly well

# Hour values are between 0 (12 AM/midnight) and 23 (11 PM).  The dates are in UTC, which is about 7 hours ahead of CA time.
# For example, if you wanted to only place trades between 6 AM and 12 PM CA time, the start hour should be 13 (1 PM UTC) and
# the end hour should be 19 (7 PM UTC).  If you want to trade all 24 hours of the day, you would set the start hour to 0 and
# the end hour to 23.
start_hours = [0, 13, 14]
end_hours = [19, 20, 23]
use_trailing_sl_vals = [True]
wait_for_close_vals = [False]

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for use_pullback in use_pullback_vals:
                    for pips_to_risk in pips_to_risk_vals:
                        for pullback_percentage in pullback_percentages:
                            for ema in ema_vals:
                                for adx_cutoff in adx_cutoffs:
                                    for start_hour in start_hours:
                                        for end_hour in end_hours:
                                            for use_trailing_sl in use_trailing_sl_vals:
                                                for wait_for_close in wait_for_close_vals:
                                                    all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pips_to_risk = None
best_pullback_percentage = None
best_ema = None
best_adx_cutoff = None
best_start_hour = None
best_end_hour = None
best_use_trailing_sl_val = None
best_wait_for_close_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'ema': ema, 'adx_cutoff': adx_cutoff, 'start_hour': start_hour, 'end_hour': end_hour, 'use_trailing_sl': use_trailing_sl, 'wait_for_close': wait_for_close})


    if total_profit > best_reward:
        best_reward = total_profit
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_ema = ema
        best_adx_cutoff = adx_cutoff
        best_start_hour = start_hour
        best_end_hour = end_hour
        best_use_trailing_sl_val = use_trailing_sl
        best_wait_for_close_val = wait_for_close

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 864

0 0 0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 863
Best reward so far: 0

-92318.69999999958 -16.010240000000003 -92334.71023999958
Num buys: 16
Num sells: 16
Num trades: 32
Num wins: 2
Num losses: 27
Win streak: 1
Loss streak: 19
Avg pips risked: 0.2999999999999994
Avg win amount: 15831.449999999957
Min win amount: 4650.299999999956
Max win amount: 27012.59999999996
Avg loss amount: -4132.719999999982
Min loss amount: -4916.700000000186
Max loss amount: 0.0
Remaining runs: 862
Best reward so far: 0

-248818.39999999467 -8.4909 -248826.89089999467
Num buys: 43
Num sells: 43
Num trades: 86
Num wins: 4
Num losses: 78
Win streak: 1
Loss streak: 33
Avg pips risked: 0.1999999999999974
Avg win amount: 27925.449999999997
Min win amount: 4436.000000000063
Max win amount: 58037.19999999987
Avg loss amount: -4450.866666666601
Min loss amount: -4916.80000000007
Max loss amount: 0.0
Remaining runs: 861
Best reward 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best invert val: ' + str(best_invert_val))
print('Best ema: ' + str(best_ema))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best start hour: ' + str(best_start_hour))
print('Best end hour: ' + str(best_end_hour))
print('Best use trailing sl val: ' + str(best_use_trailing_sl_val))
print('Best wait for close val: ' + str(best_wait_for_close_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

: 